In [1]:
import pandas as pd
from nltk.corpus import stopwords

In [2]:
df = pd.read_excel('../data/processed/LNB_processed.xlsx')

In [3]:
text = df.sample(1)

In [14]:
txt = text.full_text.values[0]

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krister\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
stops = set(stopwords.words('german'))

In [16]:
import spacy
nlp = spacy.load('../data/models/spacy_model_250421/')

In [177]:
stops = set(stopwords.words('german'))

def clean_text(text):
    
    finaltext = ''
    doc = nlp(text)
    
    for token in doc:
        if token.text not in stops:
            finaltext += token.lemma_
            finaltext += ' '
            
    return finaltext

test = clean_text(txt)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string

In [178]:
sample_docs = df[(df.w_count > 100) & (df.w_count < 300)].sample(1000)

cleaned_docs = [clean_text(doc) for doc in sample_docs.full_text]

In [179]:
for doc in range(0, len(cleaned_docs)):
    cleaned_docs[doc] = cleaned_docs[doc].replace('\n', '')

In [191]:
vectorizer = TfidfVectorizer(lowercase=True,
                              max_features=300,
                              max_df=0.7,
                              min_df=5,
                              ngram_range=(3,3))

vectors = vectorizer.fit_transform(cleaned_docs)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = vectors.todense()

all_keywords = []

for text in denselist:
    ct = 0
    keywords = []
    for word in text.flat:
        if word > 0:
            keywords.append(feature_names[ct])
        ct += 1
    all_keywords.append(keywords)
    
true_k = 20
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1)
model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

with open('../temp/trc_test.txt', 'w', encoding='utf8') as f:
    for i in range(true_k):
        f.write(f'Cluster {i}')
        f.write('\n')
        for ind in order_centroids[i, :10]:
            f.write(' %s' %terms[ind])
            f.write('\n')
        f.write('\n')
        f.write('\n')

<ipython-input-126-21c40ac1d16a>:3: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  model.fit(vectors)


KMeans(max_iter=100, n_clusters=10, n_init=1)